# 程序 `alchemy_data`

这个程序的主要目的是将提取之后的特征数据进行必要的重组，使之成为可以被 PyTorch Geometric (pyg) 读取的作为训练、验证、测试的数据。

但这些返回的数据事实上在实际代入模型前还会作简单的预处理。因此这个程序作的是粗处理。但实际代入模型的数据的信息量不会比现在粗处理的信息量更大。

In [1]:
import numpy as np
import os

import torch
from sdf_reader import *
from alchemy_data import *

RDKit WARNING: [20:20:42] Enabling RDKit 2019.09.1 jupyter extensions


In [2]:
dict_1409364 = sdf_to_dict("raw-sdf/dev/sdf/atom_11/1409364.sdf")

## 函数 `atom_dat_reader`

该函数读入提取后特征的原子部分，返回 pyg 的 Data 类型。

In [3]:
atom_data = atom_dat_reader(dict_1409364[0], 0)
atom_data

Data(edge_attr=[289, 7], edge_index=[2, 289], x=[17, 13], y=[])

上述打印出来的意义是矩阵的维度。

- `x` 表示的是原子特征 (顶点特征)

- `edge_attr` 表示原子之间相互的特征 (边特征)

- `edge_index` 表示的是每个原子间相互特征，其对应的两个原子的序号 (边对应的两个顶点序号)

- `y` 表示数据集所提供的目标值 (target)；在这里我们不考虑这个变量

取出其中的矩阵的方法也比较简单：

In [4]:
atom_data.edge_attr

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 2.8442, 12.6574,  1.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 4.5064,  9.3202,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        ...,
        [ 7.9830,  0.1253,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 7.9976,  0.1250,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]])

对于顶点特征 `x` 其有 17 行代表分子中有 17 个原子；而其 13 列是不随分子的改变而改变的，意义是

- 0：核电荷数

- 1:8：原子的 one-hot 编码

- 8：原子是否带有电荷

- 9：原子是否是芳香性的

- 10:13：原子的杂化方式

对于边而言也是类似的，289=17×17 表示边的数量，而 7 列不随分子的改变而改变，意义是

- 0：原子间距离

- 1：原子间核排斥能

- 2:7：键类型

## 函数 `orbital_dat_reader`

该函数读入提取后特征的轨道部分，返回 pyg 的 Data 类型。

In [5]:
orbital_data = orbital_dat_reader(dict_1409364[1], 0)
orbital_data

Data(atom_idx=[65], edge_attr=[4225, 8], edge_index=[2, 4225], x=[65, 13], y=[])

这里额外存了 `atom_idx`，它代表的是轨道中心所对应的原子的序号。

顶点特征是也是 13 列，意义是

- 0：轨道对应原子的核电荷

- 1:8：原子的 one-hot 编码

- 8：轨道作为 STO-3G 基组的指数系数 $\zeta$

- 9：轨道是否是价层轨道

- 10:13：轨道的空间取向 ($x, y, z$ 方向)

边特征 8 列，意义是

- 0：轨道对应原子间的距离

- 1：重叠积分矩阵

- 2：动能积分矩阵

- 3：核势能积分矩阵

- 4:7：偶极积分矩阵

- 7：密度初猜 (通过 $F=T+V$ 并作对角化得到)

## 类 `AlchemyData`

类 `AlchemyData` 是从硬盘读取整理过后的数据集，并返回数据的类型；它会实际地用于模型的训练与测试中。

但若硬盘没有整理后的数据集，`AlchemyData` 在实例化过程中会先生成这个数据集。生成数据集的过程我们这里不作讨论。

In [6]:
dev_atom_dataset = AlchemyData(mode='dev', net_type="atom", train_csv_path="./raw/train.csv")
dev_atom_dataset

AlchemyData(1000)

上述程序表明原子数据的训练集 (`"atom"` 与 `"dev"`) 共有 1000 个分子。事实上，我们方才计算的分子就是第 963 号训练集数据集分子：

In [7]:
for idx, i in enumerate(dev_atom_dataset):
    try:
        if np.allclose(i.x, atom_data.x):
            print(idx)
    except:
        pass

963


除此之外，由于我们引入了目标值文件 `"./raw/train.csv"`，因此我们还能看到该分子的 12 个性质的目标值：

In [8]:
dev_atom_dataset[963].y

tensor([[-1.8340, -0.0377, -1.3812, -2.3111, -0.6381, -2.3111,  1.0454, -2.3111,
         -2.3111, -2.1717,  1.4114,  0.8701]])

你看到的下述文件夹下的文件，就是处理完后的训练、5-fold 验证、测试集的数据所在硬盘上的文件了：

In [9]:
os.listdir("processed/")

['orbital-test.pt',
 'atom-valid_04.pt',
 'orbital-dev.pt',
 'orbital-valid_03.pt',
 'orbital-valid_01.pt',
 'atom-test.pt',
 'orbital-valid_02.pt',
 'atom-dev.pt',
 'atom-valid_00.pt',
 'atom-valid_03.pt',
 'orbital-valid_04.pt',
 'atom-valid_01.pt',
 'atom-valid_02.pt',
 'orbital-valid_00.pt']

其中，对于每个分子，其原子图的数据与轨道图的数据分开存在两个文件中。